# For each enumerator, create excel file with all their assigned labs to check that contact details and visit

In [1]:
# Set up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[1]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import load_workbook
from openpyxl.formatting.rule import FormulaRule
from openpyxl.styles import Font, PatternFill
import os
from lab_assignment import assign_enumerators

In [2]:
# Load datasets
existing_assignments = pd.read_csv(config.ENUMERATORS / "assignedlabs.csv")
all_enumerators = pd.read_csv(config.ENUMERATORS / "all_enumerators.csv")

In [3]:
# Merge to get enum_foldername
assignments = existing_assignments.merge(all_enumerators[["enum_id", "foldername"]], on="enum_id", how="left")

# Create vars "Contacted person (if different)" and "Contact email (if different)" (both empty for now)
assignments["Contact person (if different)"] = ""
assignments["Contact email (if different)"] = ""

# Create variable "Visited?" based on whether lab is out of sample
assignments["Visited?"] = np.where(assignments["out_of_sample"] == 1, "No", "Yes")

# Create var "Comments?" for any notes (empty for now)
assignments["Comments?"] = ""

# Rename columns
assignments = assignments.rename(columns={"Professor": "Original contact person"})
assignments = assignments.rename(columns={"Email": "Original contact email"})

In [ ]:
# Create file for each enumerator

# Columns to include in the enumerator's file
cols_to_include = [
    "labgroupid", "Lab Group", "Original contact person", 
    "Original contact email", "Visited?", "Contact person (if different)",
    "Contact email (if different)", "Comments?"
]

for enum_id, enum_data in assignments.groupby("enum_id"):

    # Get enumerator info
    id = enum_data["enum_id"].iloc[0]
    name = enum_data["foldername"].iloc[0]
    folder_name = f"{name}_data"

    # Columns to include
    labs_for_enum = enum_data[cols_to_include].copy()

    # Create excel path
    filename = os.path.join(config.SWITCHDRIVE_ROOT, folder_name, f"check_lab_assignment.xlsx")

    # Save first without formatting
    labs_for_enum.to_excel(filename, index=False)

    # Load workbook and select active sheet
    wb = load_workbook(filename)
    ws = wb.active
    ws.title = "Lab Assignments"

    # Bold header row
    for cell in ws[1]:
        cell.font = Font(bold=True)

    # Adjust column widths
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Conditional formatting for "Visited?" column
    # Define fills for conditional formatting
    green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
    red_fill   = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    visited_col_idx = cols_to_include.index("Visited?") + 1 
    visited_col_letter = ws.cell(row=1, column=visited_col_idx).column_letter
    
    # Range of data rows
    start_row = 2
    end_row = ws.max_row
    cell_range = f"{visited_col_letter}{start_row}:{visited_col_letter}{end_row}"
                           
    # Add conditional formatting rules
    ws.conditional_formatting.add(
        cell_range,
        FormulaRule(formula=[f'LOWER(TRIM({visited_col_letter}{start_row}))="yes"'], fill=green_fill)
    )
    ws.conditional_formatting.add(
        cell_range,
        FormulaRule(formula=[f'LOWER(TRIM({visited_col_letter}{start_row}))="no"'], fill=red_fill)
    )

    # Save workbook
    wb.save(filename)